In [1]:
import os
import glob
import numpy as np
import pandas as pd

In [11]:
glob.glob(os.path.join(os.path.abspath(""),"*"))


['C:\\Users\\Bartek\\Desktop\\Pokemony\\pokemony.txt',
 'C:\\Users\\Bartek\\Desktop\\Pokemony\\pokemon_202110171358.csv',
 'C:\\Users\\Bartek\\Desktop\\Pokemony\\trainers_202110171359.csv',
 'C:\\Users\\Bartek\\Desktop\\Pokemony\\Untitled.ipynb']

In [12]:
file_1 = pd.read_csv("pokemon_202110171358.csv")
file_2 = pd.read_csv("trainers_202110171359.csv")